In [4]:
import fitz
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis.gensim_models
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Downloading package punkt to /Users/stan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
file_path = 'ESG Reports/2022_Apple_ESG_Report.pdf'

In [45]:
def cleanup_text(text):
    # remove non-breaking spaces
    text = text.replace(u'\xa0', u' ')
    # remove bullet points
    text = text.replace(u'•', u'')
    # remove any non-alphanumeric, non-hyphen characters
    text = re.sub(r'[^A-Za-z0-9- ]', '', text)
    # remove words with hyphens, as they could be compound words
    text = re.sub(r'\w+-\w+', '', text)
    return text

In [46]:
# def text_process(text_str):
#     if ' ' in text_str:
#         text_str = text_str.replace(' ', ' ')
#     if '•' in text_str:
#         text_str = text_str.replace('•', ' ')
#     text_str = re.sub(' +', ' ', text_str)
#     if '-' in text_str:
#         text_str = text_str.replace('- ', '').replace(' -', '').replace(' ,', ',').replace(' .', '.')
#     text_str = text_str.strip()
#     return text_str

def text_process(text_str):
    # handle punctuation and special characters
    text_str = re.sub(r'[^\w\s]', '', text_str)
    if '-' in text_str:
        text_str = text_str.replace('- ', '').replace(' -', '').replace(' ,', ',').replace(' .', '.')
    text_str = text_str.strip()
    return text_str

In [47]:
all_text = []
stop_words = set(stopwords.words('english'))

with fitz.open(file_path) as doc:
    for page in doc:
        text = page.get_text()
        # text = cleanup_text(text)  # Apply cleanup_text here
        text = text.replace('•', ' ')
        text = text_process(text)
        text = text.replace('\n', ' ')
        text = text.replace('\u2003', ' ')
        text = text.strip()
        all_text.append(text)


lemmatizer = WordNetLemmatizer()
processed_text = []

for doc in all_text:
    tokens = word_tokenize(doc.lower())
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens if token not in string.punctuation]
    no_stops = [token for token in lemmatized if token not in stop_words]
    cleaned_doc = " ".join(no_stops)
    cleaned_doc = re.sub(r'\d+', '', cleaned_doc) # remove digits from each document
    cleaned_doc = re.sub(' +', ' ', cleaned_doc) # remove extra spaces from each document
    processed_text.append(cleaned_doc)

doc_list = [doc.split() for doc in processed_text]

In [50]:
# 1. Convert into list of tokens
doc_list = [doc.split() for doc in processed_text]

# 2. Create the Dictionary and Corpus
dictionary = corpora.Dictionary(doc_list)
corpus = [dictionary.doc2bow(doc) for doc in doc_list]

# 3. Build the LDA model
lda_model = LdaModel(corpus=corpus,  id2word=dictionary, passes=10, random_state=1)

# 4. Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
40     0.221558  0.001933       1        1  7.845024
21     0.190619 -0.107389       2        1  5.392255
60     0.155203 -0.010613       3        1  4.758187
65     0.205594 -0.126053       4        1  3.680030
47     0.222927 -0.070810       5        1  3.675776
...         ...       ...     ...      ...       ...
17    -0.143523  0.000013      96        1  0.003068
67    -0.134890 -0.005508      97        1  0.003068
68    -0.135690 -0.004024      98        1  0.003068
16    -0.132715 -0.000540      99        1  0.003068
99    -0.131060 -0.005570     100        1  0.003068

[100 rows x 5 columns], topic_info=          Term        Freq       Total  Category  logprob  loglift
0        apple  684.000000  684.000000   Default  30.0000  30.0000
39    employee  220.000000  220.000000   Default  29.0000  29.0000
89    supplier  329.000000  329.000000   Default  28.0000  28.0000
289       page  135.000000  135.000000   Default  27.0000  27.0000
202       team  159.000000  159.000000   Default  26.0000  26.0000
..         ...         ...         ...       ...      ...      ...
229   business    0.000349  137.383266  Topic100  -7.7917  -2.4925
525       safe    0.000309   28.146909  Topic100  -7.9125  -1.0280
198    support    0.000328  164.828740  Topic100  -7.8531  -2.7360
23   community    0.000330  221.765781  Topic100  -7.8455  -3.0251
4       report    0.000325  182.902964  Topic100  -7.8620  -2.8490

[7269 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2953     35  0.880390       aasp
3289     41  0.745615      aayhf
916       7  0.605152  abatement
916      49  0.201717  abatement
1935      5  0.126900       able
...     ...       ...        ...
1262     13  0.057372       zero
1262     24  0.057372       zero
1262     25  0.229489       zero
3436      2  0.693257   zimbabwe
2752      8  0.698878       zoom

[8560 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[41, 22, 61, 66, 48, 8, 11, 73, 91, 58, 87, 19, 95, 76, 96, 28, 31, 59, 70, 62, 82, 1, 37, 21, 52, 5, 65, 34, 23, 45, 47, 39, 86, 40, 2, 9, 25, 55, 43, 77, 7, 51, 33, 16, 99, 94, 79, 81, 42, 84, 64, 67, 53, 44, 93, 80, 78, 13, 14, 83, 10, 85, 12, 75, 88, 89, 90, 6, 92, 4, 3, 97, 98, 15, 72, 74, 57, 38, 46, 36, 35, 49, 50, 32, 54, 30, 56, 29, 71, 27, 60, 26, 24, 63, 20, 18, 68, 69, 17, 100])

In [ ]:
file_path = "ESG Reports/0a8d901480b1f390a3bb5e7bd6604d8b40ab7e0c.pdf"

all_text = []
with fitz.open(file_path) as doc:
    for page in doc:
        text = page.get_text()
        text = text.replace('•', ' ')
        text = text_process(text)
        text = text.replace('\n', ' ')
        text = text.replace('\u2003', ' ')
        text = text.strip()
        all_text.append(text)

In [ ]:
# 1. Convert into list of tokens
doc_list = [doc.split() for doc in all_text]

# 2. Create the Dictionary and Corpus
dictionary = corpora.Dictionary(doc_list)
corpus = [dictionary.doc2bow(doc) for doc in doc_list]

# 3. Build the LDA model
lda_model = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary, passes=10)

# 4. Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis
